# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [1711]:
import pandas as pd
import numpy as np
import glob, os
import random
pd.options.mode.chained_assignment = None

In [1712]:
!cat ../Data/ebola/sl_data/2014-10-04.csv 

cat: ../Data/ebola/sl_data/2014-10-04.csv: No such file or directory


In [1713]:
DATA_FOLDER = '' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

The sentence : "Make sure you handle all the different expressions for new cases and deaths that are used in the reports" is quite ambigious in our opinion and can be interpreted in two ways :
- Either it means that we should compute the average for all the different new cases/deaths description that makes sense
- 

### Assumptions

In order to clean the data, we made the following assumptions : 
- A person, counted in one of the three categories (i.d. confirmed, suspect, probable) for deaths and cases, canno't be counted in another category. This reasonnable assumption means that we are not counting people multiple times when computing the means.
- As told during the lab session, we consider the Totals/National field for each country as consistent with respect to the different region/cities reports

### Loading the data for each country

In [1714]:
def load_country(country_name):
    return pd.concat(map(pd.read_csv, glob.glob(r'../Data/ebola/' + country_name + '_data/*.csv')))

In [1715]:
#Load the data into a dataframe for each country
guinea_dataframe = load_country('guinea')
liberia_dataframe = load_country('liberia')
sierraleone_dataframe = load_country('sl')

#Print columns label
print('Guinea:', guinea_dataframe.columns.get_values())
print('Liberia:', liberia_dataframe.columns.get_values())
print('Sierra Leone:', sierraleone_dataframe.columns.get_values())

Guinea: ['Beyla' 'Boffa' 'Conakry' 'Coyah' 'Dabola' 'Dalaba' 'Date' 'Description'
 'Dinguiraye' 'Dubreka' 'Forecariah' 'Gueckedou' 'Kerouane' 'Kindia'
 'Kissidougou' 'Kouroussa' 'Lola' 'Macenta' 'Mzerekore' 'Nzerekore' 'Pita'
 'Siguiri' 'Telimele' 'Totals' 'Yomou']
Liberia: ['Bomi County' 'Bong County' 'Date' 'Gbarpolu County' 'Grand Bassa'
 'Grand Cape Mount' 'Grand Gedeh' 'Grand Kru' 'Lofa County'
 'Margibi County' 'Maryland County' 'Montserrado County' 'National'
 'Nimba County' 'River Gee County' 'RiverCess County' 'Sinoe County'
 'Unnamed: 18' 'Variable']
Sierra Leone: ['34 Military Hospital' 'Bo' 'Bo EMC' 'Bombali' 'Bonthe' 'Hastings-F/Town'
 'Kailahun' 'Kambia' 'Kenema' 'Kenema (IFRC)' 'Kenema (KGH)' 'Koinadugu'
 'Kono' 'Moyamba' 'National' 'Police training School'
 'Police traning School' 'Port Loko' 'Pujehun' 'Tonkolili' 'Unnamed: 18'
 'Western area' 'Western area combined' 'Western area rural'
 'Western area urban' 'date' 'variable']


### Removing duplicates

In [1716]:
print('Guinea has duplicates ?', True in guinea_dataframe.duplicated(subset=['Date', 'Description']).values)
print('Liberia has duplicates ?', True in liberia_dataframe.duplicated(subset=['Date', 'Variable']).values)
print('Sierra Leone has duplicates ?', True in sierraleone_dataframe.duplicated(subset=['date', 'variable']).values)

Guinea has duplicates ? False
Liberia has duplicates ? True
Sierra Leone has duplicates ? True


In [1717]:
liberia_dataframe = liberia_dataframe.drop_duplicates(['Date', 'Variable'])
sierraleone_dataframe = sierraleone_dataframe.drop_duplicates(['date', 'variable'])

### Selecting the interesting descriptions/variables 

In [1718]:
guinea_descriptions = set(guinea_dataframe.Description.values.tolist())
liberia_descriptions = set(liberia_dataframe.Variable.values.tolist())
sierraleone_descriptions = set(sierraleone_dataframe.variable.values.tolist())

print(len(guinea_descriptions))
print(len(liberia_descriptions))
print(len(sierraleone_descriptions))

60
45
37


In [1719]:
# All possible descriptions for guinea
print(guinea_descriptions)

# From which we decided to keep only the following descriptions
filtered_guinea_descriptions = ['New cases of confirmed', 
                                'New cases of suspects', 
                                'New cases of probables',
                                'Total deaths of confirmed',
                                'Total deaths of suspects',
                                'Total deaths of probables']

# Will be useful later
guinea_rename_dict = {'New cases of confirmed': 'guinea_new_confirmed',
                      'New cases of suspects': 'guinea_new_suspects',
                      'New cases of probables': 'guinea_new_probables',
                      'Total deaths (confirmed + probables + suspects)': 'guinea_new_deaths',
                      'Total deaths of confirmed': 'guinea_new_death_confirmed',
                      'Total deaths of suspects': 'guinea_new_death_suspects',
                      'Total deaths of probables': 'guinea_new_death_probables',
                      'New deaths registered among health workers': 'guinea_new_deaths_health_workers',
                      'New deaths registered today': 'guinea_new_deaths',
                      'New deaths registered': 'guinea_new_deaths'}

{'Number of suspects cases among health workers', 'Total cases of confirmed', 'New deaths registered today (suspects)', 'Total deaths of confirmed', 'Total PEC center today', 'Total number of male cases', 'Total deaths registered among health workers', 'Total suspected non-class cases', 'Total cases of probables', 'Number of male suspects cases', 'New cases of confirmed among health workers', 'New deaths registered today', 'Total deaths of suspects', 'Number of contacts out of track', 'New cases of probables', 'Number of death of confirmed cases among health workers', 'Number of contacts out of the track 21 days', 'Total number of exits from CTE', 'Total samples tested', 'Number of female probables cases', 'Number of contacts to follow today', 'Number of patients tested', 'Total PEC center today (suspects)', 'Number of deaths of confirmed cases among health workers', 'Total deaths of probables', 'Number of contacts followed yesterday', 'Total cases of suspects', 'New deaths registered 

For guinea, the descriptions 'New deaths registered today (probables)' and 'New deaths registered today (suspects)' or not really interesting as there are only available for one day and equal to 0. Moreover the description 'New deaths registered today (confirmed)' is also available during only one day and is a redundancy of the description 'New deaths registered today'. Thus we decided to drop those descriptions.

In [1720]:
#All possible descriptions for liberia
print(liberia_descriptions)

#From which we decided to keep only the following descriptions
filtered_liberia_descriptions = ['New case/s (confirmed)', 
                                 'New Case/s (Suspected)', 
                                 'New Case/s (Probable)',
                                 'Total death/s in confirmed cases',
                                 'Total death/s in suspected cases',
                                 'Total death/s in probable cases']

liberia_rename_dict = {'New case/s (confirmed)': 'liberia_new_confirmed',
                       'New Case/s (Suspected)': 'liberia_new_suspects',
                       'New Case/s (Probable)': 'liberia_new_probables',
                       'Total death/s in confirmed cases': 'liberia_new_deaths_confirmed',
                       'Total death/s in suspected cases': 'liberia_new_deaths_suspected',
                       'Total death/s in probable cases': 'liberia_new_deaths_probable',
                       'Newly reported deaths': 'liberia_new_deaths',
                       'Newly Reported deaths in HCW': 'liberia_new_deaths_hcw'}

{'Specimens pending for testing', 'Total Number of Confirmed Cases of Sierra Leonean Nationality', 'New case/s (confirmed)', 'Total specimens tested', 'Total Number of Confirmed Cases \n of Guinean Nationality', 'Cumulative cases among HCW', 'Contacts who completed 21 day follow-up', 'Newly Reported deaths in HCW', 'Total death/s in suspected cases', 'New Case/s (Suspected)', 'Total Number of Confirmed Cases \n of Sierra Leonean Nationality', 'Total confirmed cases', 'Newly Reported Cases in HCW', 'Total Case/s (Probable)', 'New Case/s (Probable)', 'Case Fatality Rate (CFR) - Confirmed & Probable Cases', 'Total case/s (confirmed)', 'Contacts lost to follow-up', 'Total no. currently in Treatment \n Units', 'Newly reported deaths', 'Currently under follow-up', 'Total discharges', 'Total suspected cases', 'Total Number of Confirmed Cases of Guinean Nationality', 'Cumulative CFR', 'Total death/s in confirmed, \n probable, suspected cases', 'Total death/s in confirmed, probable, suspected c

In [1721]:
#All possible descriptions for Sierra Leone
print(sierraleone_descriptions)

#From which we decided to keep only the following descriptions
filtered_sierraleone_descriptions = ['death_confirmed', 'death_probable', 'death_suspected',
                                     'new_confirmed', 'new_probable', 'new_suspected']

sierraleone_rename_dict = {'death_confirmed': 'sierraleone_death_confirmed',
                           'death_suspected': 'sierraleone_death_suspected',
                           'death_probable': 'sierraleone_death_probable',
                           'new_confirmed': 'sierraleone_new_confirmed',
                           'new_probable': 'sierraleone_new_probable',
                           'new_suspected': 'sierraleone_new_suspected'}

{'cum_confirmed', 'new_contacts', 'new_probable', 'new_positive', 'death_probable', 'total_lab_samples', 'etc_cum_discharges', 'new_noncase', 'etc_cum_deaths', 'negative_corpse', 'new_suspected', 'etc_new_discharges', 'positive_corpse', 'death_suspected', 'repeat_samples', 'etc_cum_admission', 'cum_contacts', 'cum_suspected', 'cum_probable', 'pending', 'cum_noncase', 'death_confirmed', 'new_confirmed', 'cfr', 'new_samples', 'contacts_followed', 'etc_new_admission', 'percent_seen', 'contacts_healthy', 'cum_completed_contacts', 'new_negative', 'etc_new_deaths', 'contacts_not_seen', 'etc_currently_admitted', 'contacts_ill', 'population', 'new_completed_contacts'}


### Cleaning the dataframes

In [1722]:
guinea_dataframe = guinea_dataframe[['Date', 'Description', 'Totals']]
liberia_dataframe = liberia_dataframe[['Date', 'Variable', 'National']]
sierraleone_dataframe = sierraleone_dataframe[['date', 'variable', 'National']]

In [1723]:
# Standardize the date field for each dataframe
guinea_dataframe.Date = pd.to_datetime(guinea_dataframe.Date)
liberia_dataframe.Date = pd.to_datetime(liberia_dataframe.Date)
sierraleone_dataframe.date = pd.to_datetime(sierraleone_dataframe.date)

# Keep only the insteresting variables/descriptions
guinea_dataframe = guinea_dataframe[[des in filtered_guinea_descriptions for des in guinea_dataframe.Description]]
liberia_dataframe = liberia_dataframe[[var in filtered_liberia_descriptions for var in liberia_dataframe.Variable]]
sierraleone_dataframe = sierraleone_dataframe[[var in filtered_sierraleone_descriptions for var in sierraleone_dataframe.variable]]

# Remove rows with missing value
guinea_dataframe = guinea_dataframe.dropna()
liberia_dataframe = liberia_dataframe.dropna()
sierraleone_dataframe = sierraleone_dataframe.dropna()

# Cast all values to int
guinea_dataframe.Totals = guinea_dataframe.Totals.astype(int)
liberia_dataframe.National = liberia_dataframe.National.astype(int)
sierraleone_dataframe.National = sierraleone_dataframe.National.astype(int)

# Rename all reports 
guinea_dataframe.Description = guinea_dataframe.Description.apply(lambda des: guinea_rename_dict[des]) 
liberia_dataframe.Variable = liberia_dataframe.Variable.apply(lambda var: liberia_rename_dict[var])
sierraleone_dataframe.variable = sierraleone_dataframe.variable.apply(lambda var: sierraleone_rename_dict[var])


#### Cleaning Guinea dataframe

In [1724]:
def clean_cum_guinea(des_name, df):
    r_df = df.copy()
    guinea_parts_to_clean = r_df[r_df.Description == des_name]
    guinea_parts_to_clean = guinea_parts_to_clean.set_index(['Date', 'Description']).sort_index()
    
    cleaned_guinea_parts = (guinea_parts_to_clean - guinea_parts_to_clean.shift(1))
    
    idx = pd.IndexSlice
    r_df = r_df.set_index(['Date', 'Description']).sort_index()
    r_df.loc[idx[:, [des_name]], :] = cleaned_guinea_parts
    r_df = r_df.dropna().reset_index()
    return r_df
    
guinea_dataframe = clean_cum_guinea('guinea_new_death_confirmed', guinea_dataframe)
guinea_dataframe = clean_cum_guinea('guinea_new_death_suspects', guinea_dataframe)
guinea_dataframe = clean_cum_guinea('guinea_new_death_probables', guinea_dataframe)

#### Cleaning Liberia dataframe

We can see, from the liberia datafame below, that some rows at the end are dirty. 

In [1725]:
dirty_rows_liberia = liberia_dataframe[liberia_dataframe.Date > pd.datetime(2014, 12, 2)]
dirty_rows_liberia = dirty_rows_liberia.set_index(['Date', 'Variable']).sort_index()

cleaned_rows_liberia = dirty_rows_liberia - dirty_rows_liberia.shift(3)

liberia_dataframe = liberia_dataframe.set_index(['Date', 'Variable']).sort_index()
liberia_dataframe.loc[pd.datetime(2014, 12, 4):, :] = cleaned_rows_liberia
liberia_dataframe = liberia_dataframe.dropna().reset_index()

In [1726]:
def clean_cum_liberia(des_name, df):
    r_df = df.copy()
    liberia_parts_to_clean = r_df[r_df.Variable == des_name]
    liberia_parts_to_clean = liberia_parts_to_clean.set_index(['Date', 'Variable']).sort_index()
    
    cleaned_liberia_parts = (liberia_parts_to_clean - liberia_parts_to_clean.shift(1))
    
    idx = pd.IndexSlice
    r_df = r_df.set_index(['Date', 'Variable']).sort_index()
    r_df.loc[idx[:, [des_name]], :] = cleaned_liberia_parts
    r_df = r_df.dropna().reset_index()
    return r_df

liberia_dataframe = clean_cum_liberia('liberia_new_deaths_confirmed', liberia_dataframe)
liberia_dataframe = clean_cum_liberia('liberia_new_deaths_suspected', liberia_dataframe)
liberia_dataframe = clean_cum_liberia('liberia_new_deaths_probable', liberia_dataframe)

#### Cleaning Sierra Leone dataframe

In [1727]:
sierraleone_dataframe_copy = sierraleone_dataframe.copy().set_index(['date', 'variable']).unstack()
tmp = sierraleone_dataframe.set_index(['date', 'variable']).unstack()
tmp = tmp.National[['sierraleone_death_confirmed', 'sierraleone_death_probable', 'sierraleone_death_suspected']]
cleaned = (tmp - tmp.shift(1)).fillna(value=0)

sierraleone_dataframe_copy.National.loc[1:, ['sierraleone_death_confirmed', 'sierraleone_death_probable', 'sierraleone_death_suspected']] = cleaned[1:]
sierraleone_dataframe = sierraleone_dataframe_copy.stack().reset_index()[3:]
#tmp.National[['sierraleone_death_confirmed', 'sierraleone_death_probable', 'sierraleone_death_suspected']].loc[[pd.datetime(2014, 8, 12)]] = [pd.datetime(2014, 8, 12), np.NaN, np.NaN, np.NaN]


In [1728]:
#Remove first three lines 
sierraleone_dataframe = sierraleone_dataframe[3:]

In [1729]:
#Keeps only the year and month of each report
guinea_dataframe.Date = guinea_dataframe.Date.dt.to_period('M')
liberia_dataframe.Date = liberia_dataframe.Date.dt.to_period('M')
sierraleone_dataframe.date = sierraleone_dataframe.date.dt.to_period('M')

In [1730]:
#Computes average for each month
averages_guinea_dataframe = pd.DataFrame(guinea_dataframe.groupby(['Date', 'Description'])['Totals'].mean())
averages_liberia_dataframe = pd.DataFrame(liberia_dataframe.groupby(['Date', 'Variable'])['National'].mean())
averages_sierraleone_dataframe = pd.DataFrame(sierraleone_dataframe.groupby(['date', 'variable'])['National'].mean())

#Remove index
averages_guinea_dataframe = averages_guinea_dataframe.reset_index()
averages_liberia_dataframe = averages_liberia_dataframe.reset_index()
averages_sierraleone_dataframe = averages_sierraleone_dataframe.reset_index()

#Rename columns
averages_guinea_dataframe.columns = ['date', 'report', 'average']
averages_liberia_dataframe.columns = ['date', 'report', 'average']
averages_sierraleone_dataframe.columns = ['date', 'report', 'average']

#Stack all averages into one dataframe
all_averages_dataframe = pd.concat([averages_guinea_dataframe, averages_liberia_dataframe, averages_sierraleone_dataframe])

#Sort final dataframe, set index and unstack result (for readibility reasons)
all_averages_dataframe = all_averages_dataframe.sort_values(['date']).set_index(['date', 'report']).unstack()


In [1731]:
all_averages_dataframe

average                             \
report  guinea_new_confirmed guinea_new_death_confirmed   
date                                                      
2014-06                  NaN                        NaN   
2014-07                  NaN                        NaN   
2014-08                 12.4                     27.250   
2014-09                 13.0                     12.625   
2014-10                  6.0                     23.000   
2014-11                  NaN                        NaN   
2014-12                  NaN                        NaN   

                                                              \
report  guinea_new_death_probables guinea_new_death_suspects   
date                                                           
2014-06                        NaN                       NaN   
2014-07                        NaN                       NaN   
2014-08                     4.2500                  0.000000   
2014-09                     1.5625                 11.133333   
2014-10                     2.0000               -169.000000   
2014-11                        NaN                       NaN   
2014-12                        NaN                       NaN   

                                                                        \
report  guinea_new_probables guinea_new_suspects liberia_new_confirmed   
date                                                                     
2014-06                  NaN                 NaN              2.142857   
2014-07                  NaN                 NaN              1.818182   
2014-08               2.0000             11.8000              5.444444   
2014-09               1.1875              5.4375              6.166667   
2014-10               0.0000             28.0000              1.619048   
2014-11                  NaN                 NaN              7.800000   
2014-12                  NaN                 NaN             12.714286   

                                                                  \
report  liberia_new_deaths_confirmed liberia_new_deaths_probable   
date                                                               
2014-06                     3.000000                    0.666667   
2014-07                     3.636364                    4.545455   
2014-08                    28.333333                   32.222222   
2014-09                    26.391304                   14.130435   
2014-10                    36.833333                    9.600000   
2014-11                 -1149.000000                 -723.000000   
2014-12                          NaN                         NaN   

                                                            \
report  liberia_new_deaths_suspected liberia_new_probables   
date                                                         
2014-06                     1.833333              1.142857   
2014-07                     0.545455              3.727273   
2014-08                    21.666667             19.777778   
2014-09                    10.478261             29.333333   
2014-10                    12.200000             17.760000   
2014-11                  -516.000000              8.076923   
2014-12                          NaN              2.857143   

                                                          \
report  liberia_new_suspects sierraleone_death_confirmed   
date                                                       
2014-06             2.428571                         NaN   
2014-07             3.000000                         NaN   
2014-08            12.000000                    7.235294   
2014-09            28.333333                    6.037037   
2014-10            26.440000                   18.571429   
2014-11            19.461538                   22.421053   
2014-12             5.571429                   53.000000   

                                                                \
report  sierraleone_death_probable sierraleone_death_suspected   
date         

#### Cleaning the date columns

jk

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here